In [1]:
import json
import time
import sys
import pandas as pd
import numpy  as np
import scipy.sparse as sp
import pickle as pkl
import collections 
import gc

In [2]:
with open('../data/index_item_map.pkl', 'rb') as f:
    data_map = pkl.load(f)

In [3]:
paper_id_title = data_map['paper_id_title']
author_id_name = data_map['author_id_name']
venue_id_name = data_map['venue_id_name']
keywords_id_name = data_map['keywords_id_name']
paper_title_id = data_map['paper_title_id']
author_name_id = data_map['author_name_id']
venue_name_id = data_map['venue_name_id']
keywords_name_id = data_map['keywords_name_id']
keywords_set = data_map['keywords_set']
venue_set = data_map['venue_set']

In [4]:
with open ('../data/cumulative_labels.pkl','rb') as f:
    cumulative_labels = pkl.load(f)

In [5]:
full_pair = pd.read_csv('../data/whole_list.txt',names=['P','O','type'])

In [6]:
def construct_index_from_one_df(df,item_id,flag):
    if flag == 'P':
        max_len = 100
    elif flag == 'A':
        max_len = 20
    elif flag == 'V':
        max_len = 1
    elif flag == 'K':
        max_len = 15
    dic = collections.defaultdict(lambda : [])
    keys = list(df['P'])
    values = list(df['O'])
    for i in  range(len(keys)):
        try:
            if len(dic[keys[i]])<max_len:
                index_id = item_id[values[i]]
                dic[keys[i]].append(index_id)
            cnt = cnt + 1
        except:
            pass
    for i in range(len(keys)):
        need_len = max_len-len(dic[keys[i]])
        for j in range(need_len):
            dic[keys[i]].append(-1)
    return dic


In [7]:
def construct_index_5_years(df,item_list,flag):
    dic_1 = construct_index_from_one_df(df,item_list[0],flag)
    dic_2 = construct_index_from_one_df(df,item_list[1],flag)
    dic_3 = construct_index_from_one_df(df,item_list[2],flag)
    dic_4 = construct_index_from_one_df(df,item_list[3],flag)
    dic_5 = construct_index_from_one_df(df,item_list[4],flag)
    return [dic_1,dic_2,dic_3,dic_4,dic_5]

In [8]:
def index_mapping_one(paper_lst,id_dic):
    return [id_dic[paper] for paper in paper_lst]
def index_mapping_five(paper_lst,id_dic_list):
    return np.array([index_mapping_one(paper_lst,id_dic) for id_dic in id_dic_list])

In [9]:
with open('../individual_data/graph_2000.pkl','rb') as f:
    graph_2000 = pkl.load(f)
item_id_2000 = graph_2000['item_id']
with open('../individual_data/graph_2001.pkl','rb') as f:
    graph_2001 = pkl.load(f)
item_id_2001 = graph_2001['item_id']
with open('../individual_data/graph_2002.pkl','rb') as f:
    graph_2002 = pkl.load(f)
item_id_2002 = graph_2002['item_id']
with open('../individual_data/graph_2003.pkl','rb') as f:
    graph_2003 = pkl.load(f)
item_id_2003 = graph_2003['item_id']
with open('../individual_data/graph_2004.pkl','rb') as f:
    graph_2004 = pkl.load(f)
item_id_2004 = graph_2004['item_id']

In [10]:
t = time.time()
P1Y_2005 = full_pair[full_pair.O==2005]
all_paper_2005 = pd.DataFrame({"P":list(set(P1Y_2005['P']))})
full_pair_2005 = pd.merge(all_paper_2005,full_pair,how="left")
full_pair_2005_P1P = full_pair_2005[full_pair_2005.type=='P1P']
full_pair_2005_P1A = full_pair_2005[full_pair_2005.type=='P1A']
full_pair_2005_P1V = full_pair_2005[full_pair_2005.type=='P1V']
full_pair_2005_P1K = full_pair_2005[full_pair_2005.type=='P1K']
graph_lookip_list = [item_id_2000,item_id_2001,item_id_2002,item_id_2003,item_id_2004]
P2005_1_P1P_list = construct_index_5_years(full_pair_2005_P1P,graph_lookip_list,'P')
P2005_1_P1A_list = construct_index_5_years(full_pair_2005_P1A,graph_lookip_list,'A')
P2005_1_P1V_list = construct_index_5_years(full_pair_2005_P1V,graph_lookip_list,'V')
P2005_1_P1K_list = construct_index_5_years(full_pair_2005_P1K,graph_lookip_list,'K')
P2005_label_cumu = list(cumulative_labels['P2005_label']['P'])
P2005_1_P1P_list = index_mapping_five(P2005_label_cumu,P2005_1_P1P_list)
P2005_1_P1A_list = index_mapping_five(P2005_label_cumu,P2005_1_P1A_list)
P2005_1_P1V_list = index_mapping_five(P2005_label_cumu,P2005_1_P1V_list)
P2005_1_P1K_list = index_mapping_five(P2005_label_cumu,P2005_1_P1K_list)
print ((P2005_1_P1P_list).shape)
print ((P2005_1_P1A_list).shape)
print ((P2005_1_P1V_list).shape)
print ((P2005_1_P1K_list).shape)
with open('../individual_data/index_2005.pkl','wb') as f:
    pkl.dump([P2005_1_P1P_list,P2005_1_P1A_list,P2005_1_P1V_list,P2005_1_P1K_list],f,0)
time.time()-t

(5, 105600, 100)
(5, 105600, 20)
(5, 105600, 1)
(5, 105600, 15)


42.06094455718994

In [11]:
with open('../individual_data/graph_2005.pkl','rb') as f:
    graph_2005 = pkl.load(f)
item_id_2005 = graph_2005['item_id']

In [12]:
t = time.time()
P1Y_2006 = full_pair[full_pair.O==2006]
all_paper_2006 = pd.DataFrame({"P":list(set(P1Y_2006['P']))})
full_pair_2006 = pd.merge(all_paper_2006,full_pair,how="left")
full_pair_2006_P1P = full_pair_2006[full_pair_2006.type=='P1P']
full_pair_2006_P1A = full_pair_2006[full_pair_2006.type=='P1A']
full_pair_2006_P1V = full_pair_2006[full_pair_2006.type=='P1V']
full_pair_2006_P1K = full_pair_2006[full_pair_2006.type=='P1K']
graph_lookip_list = [item_id_2001,item_id_2002,item_id_2003,item_id_2004,item_id_2005]
P2006_1_P1P_list = construct_index_5_years(full_pair_2006_P1P,graph_lookip_list,'P')
P2006_1_P1A_list = construct_index_5_years(full_pair_2006_P1A,graph_lookip_list,'A')
P2006_1_P1V_list = construct_index_5_years(full_pair_2006_P1V,graph_lookip_list,'V')
P2006_1_P1K_list = construct_index_5_years(full_pair_2006_P1K,graph_lookip_list,'K')
P2006_label_cumu = list(cumulative_labels['P2006_label']['P'])
P2006_1_P1P_list = index_mapping_five(P2006_label_cumu,P2006_1_P1P_list)
P2006_1_P1A_list = index_mapping_five(P2006_label_cumu,P2006_1_P1A_list)
P2006_1_P1V_list = index_mapping_five(P2006_label_cumu,P2006_1_P1V_list)
P2006_1_P1K_list = index_mapping_five(P2006_label_cumu,P2006_1_P1K_list)
print ((P2006_1_P1P_list).shape)
print ((P2006_1_P1A_list).shape)
print ((P2006_1_P1V_list).shape)
print ((P2006_1_P1K_list).shape)
with open('../individual_data/index_2006.pkl','wb') as f:
    pkl.dump([P2006_1_P1P_list,P2006_1_P1A_list,P2006_1_P1V_list,P2006_1_P1K_list],f,0)
time.time()-t

(5, 122190, 100)
(5, 122190, 20)
(5, 122190, 1)
(5, 122190, 15)


48.8684561252594

In [13]:
with open('../individual_data/graph_2006.pkl','rb') as f:
    graph_2006 = pkl.load(f)
item_id_2006 = graph_2006['item_id']

In [14]:
t = time.time()
P1Y_2007 = full_pair[full_pair.O==2007]
all_paper_2007 = pd.DataFrame({"P":list(set(P1Y_2007['P']))})
full_pair_2007 = pd.merge(all_paper_2007,full_pair,how="left")
full_pair_2007_P1P = full_pair_2007[full_pair_2007.type=='P1P']
full_pair_2007_P1A = full_pair_2007[full_pair_2007.type=='P1A']
full_pair_2007_P1V = full_pair_2007[full_pair_2007.type=='P1V']
full_pair_2007_P1K = full_pair_2007[full_pair_2007.type=='P1K']
graph_lookip_list = [item_id_2002,item_id_2003,item_id_2004,item_id_2005,item_id_2006]
P2007_1_P1P_list = construct_index_5_years(full_pair_2007_P1P,graph_lookip_list,'P')
P2007_1_P1A_list = construct_index_5_years(full_pair_2007_P1A,graph_lookip_list,'A')
P2007_1_P1V_list = construct_index_5_years(full_pair_2007_P1V,graph_lookip_list,'V')
P2007_1_P1K_list = construct_index_5_years(full_pair_2007_P1K,graph_lookip_list,'K')
P2007_label_cumu = list(cumulative_labels['P2007_label']['P'])
P2007_1_P1P_list = index_mapping_five(P2007_label_cumu,P2007_1_P1P_list)
P2007_1_P1A_list = index_mapping_five(P2007_label_cumu,P2007_1_P1A_list)
P2007_1_P1V_list = index_mapping_five(P2007_label_cumu,P2007_1_P1V_list)
P2007_1_P1K_list = index_mapping_five(P2007_label_cumu,P2007_1_P1K_list)
print ((P2007_1_P1P_list).shape)
print ((P2007_1_P1A_list).shape)
print ((P2007_1_P1V_list).shape)
print ((P2007_1_P1K_list).shape)
with open('../individual_data/index_2007.pkl','wb') as f:
    pkl.dump([P2007_1_P1P_list,P2007_1_P1A_list,P2007_1_P1V_list,P2007_1_P1K_list],f,0)
time.time()-t

(5, 134863, 100)
(5, 134863, 20)
(5, 134863, 1)
(5, 134863, 15)


55.13737082481384

In [15]:
with open('../individual_data/graph_2007.pkl','rb') as f:
    graph_2007 = pkl.load(f)
item_id_2007 = graph_2007['item_id']

In [16]:
t = time.time()
P1Y_2008 = full_pair[full_pair.O==2008]
all_paper_2008 = pd.DataFrame({"P":list(set(P1Y_2008['P']))})
full_pair_2008 = pd.merge(all_paper_2008,full_pair,how="left")
full_pair_2008_P1P = full_pair_2008[full_pair_2008.type=='P1P']
full_pair_2008_P1A = full_pair_2008[full_pair_2008.type=='P1A']
full_pair_2008_P1V = full_pair_2008[full_pair_2008.type=='P1V']
full_pair_2008_P1K = full_pair_2008[full_pair_2008.type=='P1K']
graph_lookip_list = [item_id_2003,item_id_2004,item_id_2005,item_id_2006,item_id_2007]
P2008_1_P1P_list = construct_index_5_years(full_pair_2008_P1P,graph_lookip_list,'P')
P2008_1_P1A_list = construct_index_5_years(full_pair_2008_P1A,graph_lookip_list,'A')
P2008_1_P1V_list = construct_index_5_years(full_pair_2008_P1V,graph_lookip_list,'V')
P2008_1_P1K_list = construct_index_5_years(full_pair_2008_P1K,graph_lookip_list,'K')
P2008_label_cumu = list(cumulative_labels['P2008_label']['P'])
P2008_1_P1P_list = index_mapping_five(P2008_label_cumu,P2008_1_P1P_list)
P2008_1_P1A_list = index_mapping_five(P2008_label_cumu,P2008_1_P1A_list)
P2008_1_P1V_list = index_mapping_five(P2008_label_cumu,P2008_1_P1V_list)
P2008_1_P1K_list = index_mapping_five(P2008_label_cumu,P2008_1_P1K_list)
print ((P2008_1_P1P_list).shape)
print ((P2008_1_P1A_list).shape)
print ((P2008_1_P1V_list).shape)
print ((P2008_1_P1K_list).shape)
with open('../individual_data/index_2008.pkl','wb') as f:
    pkl.dump([P2008_1_P1P_list,P2008_1_P1A_list,P2008_1_P1V_list,P2008_1_P1K_list],f,0)
time.time()-t

(5, 142965, 100)
(5, 142965, 20)
(5, 142965, 1)
(5, 142965, 15)


58.42009711265564

In [17]:
with open('../individual_data/graph_2008.pkl','rb') as f:
    graph_2008 = pkl.load(f)
item_id_2008 = graph_2008['item_id']

In [18]:
t = time.time()
P1Y_2009 = full_pair[full_pair.O==2009]
all_paper_2009 = pd.DataFrame({"P":list(set(P1Y_2009['P']))})
full_pair_2009 = pd.merge(all_paper_2009,full_pair,how="left")
full_pair_2009_P1P = full_pair_2009[full_pair_2009.type=='P1P']
full_pair_2009_P1A = full_pair_2009[full_pair_2009.type=='P1A']
full_pair_2009_P1V = full_pair_2009[full_pair_2009.type=='P1V']
full_pair_2009_P1K = full_pair_2009[full_pair_2009.type=='P1K']
graph_lookip_list = [item_id_2004,item_id_2005,item_id_2006,item_id_2007,item_id_2008]
P2009_1_P1P_list = construct_index_5_years(full_pair_2009_P1P,graph_lookip_list,'P')
P2009_1_P1A_list = construct_index_5_years(full_pair_2009_P1A,graph_lookip_list,'A')
P2009_1_P1V_list = construct_index_5_years(full_pair_2009_P1V,graph_lookip_list,'V')
P2009_1_P1K_list = construct_index_5_years(full_pair_2009_P1K,graph_lookip_list,'K')
P2009_label_cumu = list(cumulative_labels['P2009_label']['P'])
P2009_1_P1P_list = index_mapping_five(P2009_label_cumu,P2009_1_P1P_list)
P2009_1_P1A_list = index_mapping_five(P2009_label_cumu,P2009_1_P1A_list)
P2009_1_P1V_list = index_mapping_five(P2009_label_cumu,P2009_1_P1V_list)
P2009_1_P1K_list = index_mapping_five(P2009_label_cumu,P2009_1_P1K_list)
print ((P2009_1_P1P_list).shape)
print ((P2009_1_P1A_list).shape)
print ((P2009_1_P1V_list).shape)
print ((P2009_1_P1K_list).shape)
with open('../individual_data/index_2009.pkl','wb') as f:
    pkl.dump([P2009_1_P1P_list,P2009_1_P1A_list,P2009_1_P1V_list,P2009_1_P1K_list],f,0)
time.time()-t

(5, 158773, 100)
(5, 158773, 20)
(5, 158773, 1)
(5, 158773, 15)


65.55812406539917

In [19]:
with open('../individual_data/graph_2009.pkl','rb') as f:
    graph_2009 = pkl.load(f)
item_id_2009 = graph_2009['item_id']

In [20]:
t = time.time()
P1Y_2010 = full_pair[full_pair.O==2010]
all_paper_2010 = pd.DataFrame({"P":list(set(P1Y_2010['P']))})
full_pair_2010 = pd.merge(all_paper_2010,full_pair,how="left")
full_pair_2010_P1P = full_pair_2010[full_pair_2010.type=='P1P']
full_pair_2010_P1A = full_pair_2010[full_pair_2010.type=='P1A']
full_pair_2010_P1V = full_pair_2010[full_pair_2010.type=='P1V']
full_pair_2010_P1K = full_pair_2010[full_pair_2010.type=='P1K']
graph_lookip_list = [item_id_2005,item_id_2006,item_id_2007,item_id_2008,item_id_2009]
P2010_1_P1P_list = construct_index_5_years(full_pair_2010_P1P,graph_lookip_list,'P')
P2010_1_P1A_list = construct_index_5_years(full_pair_2010_P1A,graph_lookip_list,'A')
P2010_1_P1V_list = construct_index_5_years(full_pair_2010_P1V,graph_lookip_list,'V')
P2010_1_P1K_list = construct_index_5_years(full_pair_2010_P1K,graph_lookip_list,'K')
P2010_label_cumu = list(cumulative_labels['P2010_label']['P'])
P2010_1_P1P_list = index_mapping_five(P2010_label_cumu,P2010_1_P1P_list)
P2010_1_P1A_list = index_mapping_five(P2010_label_cumu,P2010_1_P1A_list)
P2010_1_P1V_list = index_mapping_five(P2010_label_cumu,P2010_1_P1V_list)
P2010_1_P1K_list = index_mapping_five(P2010_label_cumu,P2010_1_P1K_list)
print ((P2010_1_P1P_list).shape)
print ((P2010_1_P1A_list).shape)
print ((P2010_1_P1V_list).shape)
print ((P2010_1_P1K_list).shape)
with open('../individual_data/index_2010.pkl','wb') as f:
    pkl.dump([P2010_1_P1P_list,P2010_1_P1A_list,P2010_1_P1V_list,P2010_1_P1K_list],f,0)
time.time()-t

(5, 164824, 100)
(5, 164824, 20)
(5, 164824, 1)
(5, 164824, 15)


68.50755572319031

In [13]:
with open('../individual_data/graph_2010.pkl','rb') as f:
    graph_2010 = pkl.load(f)
item_id_2010 = graph_2010['item_id']

In [14]:
t = time.time()
P1Y_2011 = full_pair[full_pair.O==2011]
all_paper_2011 = pd.DataFrame({"P":list(set(P1Y_2011['P']))})
full_pair_2011 = pd.merge(all_paper_2011,full_pair,how="left")
full_pair_2011_P1P = full_pair_2011[full_pair_2011.type=='P1P']
full_pair_2011_P1A = full_pair_2011[full_pair_2011.type=='P1A']
full_pair_2011_P1V = full_pair_2011[full_pair_2011.type=='P1V']
full_pair_2011_P1K = full_pair_2011[full_pair_2011.type=='P1K']
graph_lookip_list = [item_id_2006,item_id_2007,item_id_2008,item_id_2009,item_id_2010]
P2011_1_P1P_list = construct_index_5_years(full_pair_2011_P1P,graph_lookip_list,'P')
P2011_1_P1A_list = construct_index_5_years(full_pair_2011_P1A,graph_lookip_list,'A')
P2011_1_P1V_list = construct_index_5_years(full_pair_2011_P1V,graph_lookip_list,'V')
P2011_1_P1K_list = construct_index_5_years(full_pair_2011_P1K,graph_lookip_list,'K')
P2011_label_cumu = list(cumulative_labels['P2011_label']['P'])
P2011_1_P1P_list = index_mapping_five(P2011_label_cumu,P2011_1_P1P_list)
P2011_1_P1A_list = index_mapping_five(P2011_label_cumu,P2011_1_P1A_list)
P2011_1_P1V_list = index_mapping_five(P2011_label_cumu,P2011_1_P1V_list)
P2011_1_P1K_list = index_mapping_five(P2011_label_cumu,P2011_1_P1K_list)
print ((P2011_1_P1P_list).shape)
print ((P2011_1_P1A_list).shape)
print ((P2011_1_P1V_list).shape)
print ((P2011_1_P1K_list).shape)
with open('../individual_data/index_2011.pkl','wb') as f:
    pkl.dump([P2011_1_P1P_list,P2011_1_P1A_list,P2011_1_P1V_list,P2011_1_P1K_list],f,0)
time.time()-t

(5, 177692, 100)
(5, 177692, 20)
(5, 177692, 1)
(5, 177692, 15)


76.28882098197937

In [15]:
with open('../data/graph_2011.pkl','rb') as f:
    graph_2011 = pkl.load(f)
item_id_2011 = graph_2011['item_id']

In [16]:
t = time.time()
P1Y_2012 = full_pair[full_pair.O==2012]
all_paper_2012 = pd.DataFrame({"P":list(set(P1Y_2012['P']))})
full_pair_2012 = pd.merge(all_paper_2012,full_pair,how="left")
full_pair_2012_P1P = full_pair_2012[full_pair_2012.type=='P1P']
full_pair_2012_P1A = full_pair_2012[full_pair_2012.type=='P1A']
full_pair_2012_P1V = full_pair_2012[full_pair_2012.type=='P1V']
full_pair_2012_P1K = full_pair_2012[full_pair_2012.type=='P1K']
graph_lookip_list = [item_id_2007,item_id_2008,item_id_2009,item_id_2010,item_id_2011]
P2012_1_P1P_list = construct_index_5_years(full_pair_2012_P1P,graph_lookip_list,'P')
P2012_1_P1A_list = construct_index_5_years(full_pair_2012_P1A,graph_lookip_list,'A')
P2012_1_P1V_list = construct_index_5_years(full_pair_2012_P1V,graph_lookip_list,'V')
P2012_1_P1K_list = construct_index_5_years(full_pair_2012_P1K,graph_lookip_list,'K')
P2012_label_cumu = list(cumulative_labels['P2012_label']['P'])
P2012_1_P1P_list = index_mapping_five(P2012_label_cumu,P2012_1_P1P_list)
P2012_1_P1A_list = index_mapping_five(P2012_label_cumu,P2012_1_P1A_list)
P2012_1_P1V_list = index_mapping_five(P2012_label_cumu,P2012_1_P1V_list)
P2012_1_P1K_list = index_mapping_five(P2012_label_cumu,P2012_1_P1K_list)
print ((P2012_1_P1P_list).shape)
print ((P2012_1_P1A_list).shape)
print ((P2012_1_P1V_list).shape)
print ((P2012_1_P1K_list).shape)
with open('../data/index_2012.pkl','wb') as f:
    pkl.dump([P2012_1_P1P_list,P2012_1_P1A_list,P2012_1_P1V_list,P2012_1_P1K_list],f,0)
time.time()-t

(5, 187915, 100)
(5, 187915, 20)
(5, 187915, 1)
(5, 187915, 15)


79.54049229621887

In [17]:
with open('../data/graph_2012.pkl','rb') as f:
    graph_2012 = pkl.load(f)
item_id_2012 = graph_2012['item_id']

In [18]:
t = time.time()
P1Y_2013 = full_pair[full_pair.O==2013]
all_paper_2013 = pd.DataFrame({"P":list(set(P1Y_2013['P']))})
full_pair_2013 = pd.merge(all_paper_2013,full_pair,how="left")
full_pair_2013_P1P = full_pair_2013[full_pair_2013.type=='P1P']
full_pair_2013_P1A = full_pair_2013[full_pair_2013.type=='P1A']
full_pair_2013_P1V = full_pair_2013[full_pair_2013.type=='P1V']
full_pair_2013_P1K = full_pair_2013[full_pair_2013.type=='P1K']
graph_lookip_list = [item_id_2008,item_id_2009,item_id_2010,item_id_2011,item_id_2012]
P2013_1_P1P_list = construct_index_5_years(full_pair_2013_P1P,graph_lookip_list,'P')
P2013_1_P1A_list = construct_index_5_years(full_pair_2013_P1A,graph_lookip_list,'A')
P2013_1_P1V_list = construct_index_5_years(full_pair_2013_P1V,graph_lookip_list,'V')
P2013_1_P1K_list = construct_index_5_years(full_pair_2013_P1K,graph_lookip_list,'K')
P2013_label_cumu = list(cumulative_labels['P2013_label']['P'])
P2013_1_P1P_list = index_mapping_five(P2013_label_cumu,P2013_1_P1P_list)
P2013_1_P1A_list = index_mapping_five(P2013_label_cumu,P2013_1_P1A_list)
P2013_1_P1V_list = index_mapping_five(P2013_label_cumu,P2013_1_P1V_list)
P2013_1_P1K_list = index_mapping_five(P2013_label_cumu,P2013_1_P1K_list)
print ((P2013_1_P1P_list).shape)
print ((P2013_1_P1A_list).shape)
print ((P2013_1_P1V_list).shape)
print ((P2013_1_P1K_list).shape)
with open('../data/index_2013.pkl','wb') as f:
    pkl.dump([P2013_1_P1P_list,P2013_1_P1A_list,P2013_1_P1V_list,P2013_1_P1K_list],f,0)
time.time()-t

(5, 197778, 100)
(5, 197778, 20)
(5, 197778, 1)
(5, 197778, 15)


84.14398670196533